In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model
from datetime import datetime
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint
import time

In [2]:
#take 1 CSV, then split it to 3..
class FeatureEngineering:

    def __init__(self, ValidationStart, ValidationEnd, trainHdfPath, trainHdfFile, testHdfPath1, testHdfPath2, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationEnd = ValidationEnd
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath1 = testHdfPath1
        self.testHdfPath2 = testHdfPath2
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        data.reset_index(inplace=True)
        data.drop("index",axis=1, inplace=True)
        #data.index = data.index.astype('uint32')
        gc.collect()
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
            
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1 = pd.read_hdf(self.testHdfPath1,self.testHdfFile)
            self.test2 = pd.read_hdf(self.testHdfPath2,self.testHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in memory'''
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth == 'both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            self.test1 = tempTest.loc[tempTest.Semana.values == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana.values == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
            
    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict,index=False)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth =='both':
                self.train.loc[:,column] =  np.apply_along_axis(func,0,FE.train[column].values).astype(localType)
                #np.apply_along_axis(lambda x: x+1,0,FE.train["Semana"]).astype("int32")
            if trainOrTestOrBoth == 'test' or trainOrTestOrBoth == 'both':
                self.test1.loc[:,column] =  np.apply_along_axis(func,0,FE.test1[column].values).astype(localType)
                self.test2.loc[:,column] =  np.apply_along_axis(func,0,FE.test2[column].values).astype(localType)
        gc.collect()
        
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t', index="False")
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.to_hdf(self.testHdfPath1, self.testHdfFile, format='t', index="False")
            self.test2.to_hdf(self.testHdfPath2, self.testHdfFile, format='t', index="False")
        
    def AddDemandaGeneralMean(self): 
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].loc[
         #   self.train.loc[:,'Semana'] < 10].mean().astype("float32")
        self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].values[
        (self.train.loc[:,'Semana'].values < self.ValidationStart).values].mean().astype("float32")
        #self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        #self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        #display(self.train)
        #del DemandaMeanWithoutLag
        gc.collect()
        
    '''ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),'''
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        tempData = self.train[self.train['Semana'].values <= (self.ValidationEnd - config.lag)]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData['Semana'].values + config.lag
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
            self.test1.loc[:,config.targetVariable] = np.nan
            
            if config.lag != 1:
                self.test2.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)            
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].copy().groupby(groups).agg(aggregate[0])
                gc.collect()
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                self.test1 = self.test1.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                if config.lag != 1:
                    self.test2 = self.test2.merge( groupedDataframe, left_on=groups,
                        right_index=True, how='left', sort=False,copy=False)
                
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            display(self.test1)
            display(self.test2)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable].values), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable].values)
                    , name].values
                self.test1.loc[pd.isnull(self.test1[config.targetVariable].values), 
                    config.targetVariable] = self.test1.loc[pd.isnull(self.test1[config.targetVariable].values),
                    name].values
                if config.lag != 1:
                    self.test2.loc[pd.isnull(self.test2[config.targetVariable].values), 
                        config.targetVariable] = self.test2.loc[pd.isnull(self.test2[config.targetVariable].values)
                        , name].values
                    
                count = self.test1[config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part 1 in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                if config.lag != 1:
                    count = self.test2.loc[:,config.targetVariable].isnull().sum()
                    print "Count of missing numbers after {} in validation part 2 in column {} is {}".format(name, 
                        config.targetVariable,str(count))
                
                
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                    self.test1.drop(name, axis=1, inplace=True)
                    if config.lag != 1:
                        self.test2.drop(name, axis=1, inplace=True)
                gc.collect()
                #Only in tesst
                #if count == 0:
                 #   break
        del tempData
        display(self.train)   
        display(self.test1)   
        display(self.test2)
        gc.collect()
        return 
    
    def DeleteLaggedWeeksFromTrain(self,trainOrTestOrBoth):
        self.train = self.train[self.train['Semana'].values >= (3 + self.maxLag)]
        gc.collect()
        display(self.train.head(2))
        
    def ReadFirstNRowsOfACsv(self, nrows, trainOrTestOrBoth) :
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes, nrows = nrows)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes, nrows = nrows*2)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
    
    #Use when concatanating train and validation before predict test for example..
    def AppendTestToTrain(self,deleteTest = True):
        self.train = self.train.append(self.test1,ignore_index=True)
        gc.collect()
        if(deleteTest):
            del self.test1
            gc.collect()
        try:
            self.train = self.train.append(self.test2,ignore_index=True)
            gc.collect()
            if(deleteTest):
                del self.test2
                gc.collect()
        except:
            pass
        #BAD PERFORMANCE!!
    #Split train data to train and test1 and test2 (validation)
    #def SplitTrainToTestUsingValidationStart(self):
     #   boolCondition = self.train.Semana == self.ValidationStart
      #  self.test1 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        
       # boolCondition = self.train.Semana == self.ValidationEnd
       # self.test2 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
      #  del boolCondition
      #  gc.collect()
    
    #Reaches 3x memory from train, because of test1, test2 and train itself at the end.. GC fixed in the end..
    def SplitTrainToTestUsingValidationStart(self):
        boolCondition = self.train.Semana.values == self.ValidationStart
        self.test1 = self.train[boolCondition]
        boolCondition = self.train.Semana.values == self.ValidationEnd
        self.test2 = self.train[boolCondition]
        FE.train = FE.train[ FE.train.Semana.values < FE.ValidationStart ]
        del boolCondition
        gc.collect()

In [9]:
parameterDict =       {"ValidationStart":10, 
 "ValidationEnd":11,
   "maxLag":2,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/test1.h5",
    "testHdfPath2":"../../input/test2.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train.csv', 'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'testHdfPath1': '../../input/test1.h5', 'ValidationEnd': 11, 'testHdfPath2': '../../input/test2.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 10, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5'}


In [10]:
FE.ReadCsv('test')

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,2,10,2045,1,2831,4549769,32940
1,7,10,1612,1,2837,4414012,35305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3538385 entries, 0 to 3538384
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 54.0 MB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3460866 entries, 0 to 3460865
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 52.8 MB
None


## Client's total sale

In [8]:
grouped = FE.train[["Cliente_ID","Venta_hoy","Dev_proxima"]].copy().groupby(["Cliente_ID"]).agg(["sum","count"])

In [16]:
grouped.loc[:,"Difference"] = grouped[("Venta_hoy","sum")] - grouped[("Dev_proxima","sum")]

In [19]:
(grouped["Difference"] < 0)

Cliente_ID
26            False
60            False
65            False
101           False
105           False
106           False
107           False
215           False
262           False
465           False
772           False
786           False
791           False
795           False
797           False
906           False
930            True
1080          False
1213          False
1266          False
1311          False
1436          False
1438          False
1589          False
1697          False
2003          False
2006          False
2010          False
2011          False
2012          False
              ...  
9891358       False
9891984       False
9892002       False
9892008       False
9892020       False
9892026       False
9902512       False
9902526       False
9927190       False
9943143       False
9962735       False
9982898       False
9991105       False
9999999       False
10033178      False
10142492      False
10142588      False
10281997      False
10303963 

In [20]:
grouped[grouped["Cliente_ID"].values < 0 ]

KeyError: 'Cliente_ID'

In [21]:
gc.collect()

28

In [25]:
grouped.reset_index(inplace=True)

In [26]:
grouped.columns

MultiIndex(levels=[[u'Venta_hoy', u'Dev_proxima', u'Difference', u'Cliente_ID'], [u'sum', u'count', u'']],
           labels=[[3, 0, 0, 1, 1, 2], [2, 0, 1, 0, 1, 2]])

In [27]:
grouped

Cliente_ID     Venta_hoy        Dev_proxima          Difference
                             sum count          sum count              
0               26  1.281493e+05   227  6969.720215   227  1.211795e+05
1               60  2.148014e+06   248     0.000000   248  2.148014e+06
2               65  1.113132e+06   630   197.600006   630  1.112934e+06
3              101  2.735388e+04    15     0.000000    15  2.735388e+04
4              105  3.106252e+05   345     0.000000   345  3.106252e+05
5              106  6.404854e+04   107     0.000000   107  6.404854e+04
6              107  1.297966e+05   277  6368.580078   277  1.234281e+05
7              215  2.651600e+02    10     0.000000    10  2.651600e+02
8              262  2.592200e+02     2     0.000000     2  2.592200e+02
9              465  2.903280e+03    13     0.000000    13  2.903280e+03
10             772  4.218230e+03   152    94.959999   152  4.123270e+03
11             786  9.144900e+02    53    16.760000    53  8.977300e+02
12             791  5.875400e+02    35    14.540000    35  5.730000e+02
13             795  8.588300e+02    43    16.670000    43  8.421600e+02
14             797  4.022800e+03   141     0.000000   141  4.022800e+03
15             906  2.202800e+03    56     0.000000    56  2.202800e+03
16             930  2.616400e+02     6   437.000000     6 -1.753600e+02
17            1080  2.755670e+03    25     0.000000    25  2.755670e+03
18            1213  7.781000e+01     3     0.000000     3  7.781000e+01
19            1266  4.321600e+02    13     0.000000    13  4.321600e+02
20            1311  5.141340e+03    89     0.000000    89  5.141340e+03
21            1436  8.451200e+02    31    15.010000    31  8.301100e+02
22            1438  1.579610e+03    20     0.000000    20  1.579610e+03
23            1589  1.135933e+04   121     0.000000   121  1.135933e+04
24            1697  3.876400e+02     5     0.000000     5  3.876400e+02
25            2003  1.960310e+03    77     9.630000    77  1.950680e+03
26            2006  5.236310e+03   142    34.070000   142  5.202240e+03
27            2010  4.705090e+03   150   139.610001   150  4.565480e+03
28            2011  2.319720e+03    82    28.599998    82  2.291120e+03
29            2012  2.939300e+02    14     0.000000    14  2.939300e+02
...            ...           ...   ...          ...   ...           ...
880574     9891358  2.025714e+04   225     0.000000   225  2.025714e+04
880575     9891984  2.609681e+04   261     0.000000   261  2.609681e+04
880576     9892002  2.553482e+04   238     0.000000   238  2.553482e+04
880577     9892008  1.077500e+03    24     0.000000    24  1.077500e+03
880578     9892020  3.628720e+03    63     0.000000    63  3.628720e+03
880579     9892026  2.497215e+04   178     0.000000   178  2.497215e+04
880580     9902512  1.448015e+04   178   294.019989   178  1.418613e+04
880581     9902526  3.096551e+04   229     0.000000   229  3.096551e+04
880582     9927190  4.342899e+04   334     0.000000   334  4.342899e+04
880583     9943143  4.139000e+01     3     0.000000     3  4.139000e+01
880584     9962735  1.246733e+04    55     0.000000    55  1.246733e+04
880585     9982898  4.527000e+05   370  1723.959961   370  4.509760e+05
880586     9991105  4.582690e+03    17     0.000000    17  4.582690e+03
880587     9999999  3.179801e+04   301     0.000000   301  3.179801e+04
880588    10033178  5.888240e+05   448  5137.600098   448  5.836864e+05
880589    10142492  2.101528e+04   231  1317.670044   231  1.969761e+04
880590    10142588  2.190240e+04     1     0.000000     1  2.190240e+04
880591    10281997  2.412278e+04   201   239.300003   201  2.388348e+04
880592    10303963  7.104234e+04   250  1270.079956   250  6.977227e+04
880593    10351784  2.959241e+04   202     0.000000   202  2.959241e+04
880594    10351790  3.332071e+04   207     4.400000   207  3.331631e+04
880595    10351796  3.611973e+04   218     0.000000   218  3.611973e+04
880596    10351802  3.201634e+04   176     0.000

In [28]:
FE.train[FE.train.Cliente_ID.values == 930 ]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
9016757,3,2095,11,3912,930,1373,2,61.82,0,0.0,2
9016758,3,2095,11,3912,930,42792,1,23.00,0,0.0,1
20002649,4,2095,11,3912,930,1373,2,61.82,0,0.0,2
20002650,4,2095,11,3912,930,42792,2,46.00,0,0.0,2
30704716,5,2095,11,3912,930,42792,1,23.00,0,0.0,1
40947965,6,2095,11,3912,930,42792,2,46.00,19,437.0,0


## Missing Productos..

In [13]:
missingProductIds = [32026, 37404, 37405, 32798, 32421, 37745, 36524, 37618, 35246, 33053, 46131, 32820, 37688, 36673, 37702, 35191, 32591, 37202, 42323, 48217, 32224, 98, 31203, 37610, 31211, 46064, 37617, 37362, 37620, 31655, 37494, 37495, 37496, 37626]
                     

In [23]:
missings1 = FE.test1[ np.in1d(FE.test1["Producto_ID"].values, np.array(missingProductIds))]

In [26]:
missings1.groupby("Producto_ID").count()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID
Producto_ID,,,,,,
31203,8,8,8,8,8,8
31655,1,1,1,1,1,1
32224,1,1,1,1,1,1
32421,42,42,42,42,42,42
32820,29,29,29,29,29,29
33053,1,1,1,1,1,1
35191,1,1,1,1,1,1
36524,6,6,6,6,6,6
36673,1171,1171,1171,1171,1171,1171


In [27]:
missings2 = FE.test2[ np.in1d(FE.test2["Producto_ID"].values, np.array(missingProductIds))]

In [28]:
missings2.groupby("Producto_ID").count()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID
Producto_ID,,,,,,
98,1,1,1,1,1,1
31203,8,8,8,8,8,8
31211,1,1,1,1,1,1
32026,1,1,1,1,1,1
32421,190,190,190,190,190,190
32591,1,1,1,1,1,1
32798,1,1,1,1,1,1
32820,97,97,97,97,97,97
33053,1,1,1,1,1,1


In [ ]:
8217, 32224, 98, 31203, 37610, 31211, 46064, 37617, 37362, 37620, 31655, 37494, 37495, 37496, 37626]